In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "quiz")
import numpy as np 

### 데이터셋
id; age; gender; height; weitht; ap_hi; ap_lo; cholesterol; gluc; smoke; alco; active; cardio

(cardio는 클래스, id는 데이터 인덱스)  
전체 데이터 중 60%는 학습용, 40%는 검증용으로 분리

In [6]:
data = sc.textFile("cardio_train.csv").flatMap(lambda l: l.split('\n')).map(lambda l: l.split(';'))
data = np.array(data.collect()[1:]).astype(np.float)

In [7]:
train_ratio = 0.6
train_size = int(train_ratio*len(data))
valid_size = len(data) - train_size
shuffled_idx = np.random.permutation(len(data))

In [15]:
train = data[shuffled_idx[:train_size]][:, 1:]*0.0001 # idx 열 제거
valid = data[shuffled_idx[train_size:]][:, 1:]*0.0001
print(train[:3])

train_RDD = sc.parallelize(train)
valid_RDD = sc.parallelize(valid)

[[2.2880e+00 2.0000e-04 1.7000e-02 7.3000e-03 1.3000e-02 8.0000e-03
  2.0000e-04 1.0000e-04 0.0000e+00 0.0000e+00 1.0000e-04 1.0000e-04]
 [1.5286e+00 1.0000e-04 1.7600e-02 8.1000e-03 1.1000e-02 1.7000e-02
  1.0000e-04 2.0000e-04 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00]
 [1.7582e+00 1.0000e-04 1.6500e-02 6.0000e-03 1.1000e-02 8.0000e-03
  1.0000e-04 1.0000e-04 0.0000e+00 0.0000e+00 1.0000e-04 0.0000e+00]]


In [16]:
import math

maxIter = 20
learning_rate = 0.0001
D = len(train[0])-1 #feature 

w = np.random.permutation(D).astype(np.float) # 각 feature의 가중치 벡터

for _ in range(maxIter):
    gradient = train_RDD.map(lambda p: (p[D]-1 / (1+math.exp(-np.dot(w,p[:D]))))*p[:D])\
                    .reduce(lambda a, b: a+b)
    w += learning_rate * np.array(gradient)

print("Final w: {}".format(w))

Final w: [-6.13449702  6.99888503 -0.13617787  8.93852683  3.89316787  2.92036428
  5.99886541  7.99898108  0.99992816  1.99995518  4.99933364]


### 검증

학습된 가중치와 검증 데이터셋으로 검증

In [25]:
for valid_data in valid:
    y = valid_data[D]*10000
    score = (1 / (1 + math.exp(-np.dot(w, valid_data[:D]))))*10000
    print("y = ", y, ", score = ", score)

y =  1.0 , score =  0.024512043274692516
y =  0.0 , score =  0.3771056807702082
y =  1.0 , score =  0.03860578778443125
y =  1.0 , score =  0.14355097287598606
y =  1.0 , score =  0.9950245920384844
y =  0.0 , score =  0.02550968000748582
y =  1.0 , score =  0.012604339370956879
y =  0.0 , score =  0.5781951419414144
y =  1.0 , score =  0.6866717558752103
y =  0.0 , score =  0.40145079889158714
y =  0.0 , score =  0.017272550094897404
y =  0.0 , score =  0.09585469782367999
y =  0.0 , score =  0.5710688827208461
y =  1.0 , score =  0.07368020144865546
y =  0.0 , score =  0.014724556018341535
y =  1.0 , score =  0.011682440712323732
y =  1.0 , score =  0.06660319871692759
y =  1.0 , score =  0.21745726264882292
y =  1.0 , score =  0.0361571363683292
y =  0.0 , score =  0.14100393638160874
y =  0.0 , score =  0.01633441043553995
y =  1.0 , score =  0.0164720750560058
y =  0.0 , score =  0.06772778907685616
y =  0.0 , score =  0.06250410952047959
y =  0.0 , score =  0.39720922723124413
y 